In [ ]:
%env OPENAI_API_KEY=<PUT_YOUR_API_KEY_HERE>


In [4]:
from openai import OpenAI
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

llm = ChatOpenAI(max_tokens=2048, temperature=0.5)
multiple_choice = """
Please choose an appropriate tool to answer the user's question between >>> and <<<. Just tell me the answer with the option letters A, B, C. If you think none are suitable, choose D.

>>>{question}<<<

The tools we have include:
A. A tool that can query product information and guide users in product selection
B. A tool that can query order information and obtain the latest order status
C. A tool that can search for a merchant's return and exchange policy, shipping costs, logistics duration, payment channels, and covered countries
D. None are suitable
"""
multiple_choice_prompt = PromptTemplate(template=multiple_choice, input_variables=["question"])
choice_chain = LLMChain(llm=llm, prompt=multiple_choice_prompt, output_key="answer")


In [37]:
question = "I want to buy a fingerprint door lock, but I'm not sure which brand offers the best value for money. Can you recommend one for me?"
print(choice_chain(question))


{'question': "I want to buy a fingerprint door lock, but I'm not sure which brand offers the best value for money. Can you recommend one for me?", 'answer': 'A. A tool that can query product information and guide users in product selection'}


In [38]:
question = "I have an order with the order number 20231127E12345 that I have not received yet. Could you please help me check it?"
print(choice_chain(question))


{'question': 'I have an order with the order number 20231127E12345 that I have not received yet. Could you please help me check it?', 'answer': 'B. A tool that can query order information and obtain the latest order status'}


In [40]:
question = "Can this product be delivered to Crested Butte in Colorado? How many days will it approximately take?"
print(choice_chain(question))


{'question': 'Can this product be delivered to Crested Butte in Colorado? How many days will it approximately take?', 'answer': "C. A tool that can search for a merchant's return and exchange policy, shipping costs, logistics duration, payment channels, and covered countries"}


In [41]:
question = "How has the Los Angeles Chargers performed this season?"
print(choice_chain(question))


{'question': 'How has the Los Angeles Chargers performed this season?', 'answer': 'D. None are suitable'}


In [5]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import openai

llm = openai.OpenAI(temperature=0)

def search_order(input: str) -> str:
    return "Order status: Shipped; Shipment date: 2023-11-27; Estimated delivery date: 2023-12-01"

def recommend_product(input: str) -> str:
    return "FlexBand Multifunctional Smartwatch"

def faq(intput: str) -> str:
    return "Exclusive Deals"

tools = [
    Tool(
        name = "Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(name="Recommend Product", func=recommend_product,
         description="useful for when you need to answer questions about product recommendations"
    ),
    Tool(name="FAQ", func=faq,
         description="useful for when you need to answer questions about shopping policies, like exclusive deals, return policy, shipping policy, etc."
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [6]:
question = "I want to buy a smartwatch, but I'm not sure which features would suit me. Can you help me with some recommendations?"
result = agent.run(question)
print(result)




> Entering new AgentExecutor chain...
 I need to recommend a product.
Action: Recommend Product
Action Input: Smartwatch
Observation: FlexBand Multifunctional Smartwatch
Thought: I need to know more about this product.
Action: Search Order
Action Input: FlexBand Multifunctional Smartwatch
Observation: Order status: Shipped; Shipment date: 2023-11-27; Estimated delivery date: 2023-12-01
Thought: I now know the final answer.
Final Answer: The FlexBand Multifunctional Smartwatch is available for purchase and is estimated to be delivered on December 1, 2023.

> Finished chain.
The FlexBand Multifunctional Smartwatch is available for purchase and is estimated to be delivered on December 1, 2023.


In [44]:
question = "I have an order with the order number 20231127D3478E. The order status shows as not delivered. Could you please help me check it?"
result = agent.run(question)
print(result)




> Entering new AgentExecutor chain...
 I need to find out the status of the order
Action: Search Order
Action Input: 20231127D3478E
Observation: Order status: Shipped; Shipment date: 2023-11-27; Estimated delivery date: 2023-12-01
Thought: I now know the final answer
Final Answer: Your order has been shipped on 2023-11-27 and is estimated to be delivered on 2023-12-01.

> Finished chain.
Your order has been shipped on 2023-11-27 and is estimated to be delivered on 2023-12-01.


In [45]:
question = "I am an Amazon Prime member. Can I enjoy a Prime member discount on this smartwatch?"
result = agent.run(question)
print(result)




> Entering new AgentExecutor chain...
 I need to find out if Amazon Prime members can get a discount on this product.
Action: FAQ
Action Input: Prime member discount
Observation: Exclusive Deals
Thought: I need to find out what exclusive deals are available for Prime members.
Action: FAQ
Action Input: Exclusive Deals
Observation: Exclusive Deals
Thought: I now know the final answer.
Final Answer: Yes, Amazon Prime members can enjoy exclusive deals on select products.

> Finished chain.
Yes, Amazon Prime members can enjoy exclusive deals on select products.


In [46]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", max_iterations = 2, verbose=True)
question = "I am an Amazon Prime member. Can I enjoy a Prime member discount on this smartwatch?"
result = agent.run(question)
print("===")
print(result)
print("===")




> Entering new AgentExecutor chain...
 I need to find out if Amazon Prime members can get a discount on this product.
Action: FAQ
Action Input: Prime member discount
Observation: Exclusive Deals
Thought: I need to find out what exclusive deals are available for Prime members.
Action: FAQ
Action Input: Exclusive Deals
Observation: Exclusive Deals
Thought:

> Finished chain.
===
Agent stopped due to iteration limit or time limit.
===


In [47]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import faiss
from langchain.text_splitter import SpacyTextSplitter
from langchain import VectorDBQA
from langchain.llms import openai
from langchain.document_loaders import TextLoader

llm = openai.OpenAI(temperature=0)
loader = TextLoader('./data/amazon_faq.txt')
documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="en_core_web_sm")
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = faiss.FAISS.from_documents(texts, embeddings)

faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)


/home/fw/miniconda3/envs/ai_aesthetics/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Created a chunk of size 434, which is longer than the specified 256
Created a chunk of size 466, which is longer than the specified 256
/home/fw/miniconda3/envs/ai_aesthetics/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:251: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [48]:
from langchain.agents import tool

@tool("FAQ")
def faq(intput: str) -> str:
    """"useful for when you need to answer questions about shopping policies, like exclusive deals, return policy, shipping policy, etc."""
    return faq_chain.run(intput)

tools = [
    Tool(
        name = "Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(name="Recommend Product", func=recommend_product,
         description="useful for when you need to answer questions about product recommendations"
    ),
    faq
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [49]:
question = "I am an Amazon Prime member. Can I enjoy a Prime member discount on this smartwatch?"
result = agent.run(question)
print(result)




> Entering new AgentExecutor chain...
 I need to find out if there is a discount available for Prime members.
Action: FAQ
Action Input: Prime member discount

> Entering new VectorDBQA chain...

> Finished chain.

Observation:  Selling fees and optional services may apply for Prime members. The Amazon Prime badge is available to sellers who use Fulfillment by Amazon (FBA) or Seller Fulfilled Prime (SFP) to handle logistics.
Thought: I need to find out if there is a discount available for this specific product.
Action: Search Order
Action Input: Smartwatch
Observation: Order status: Shipped; Shipment date: 2023-11-27; Estimated delivery date: 2023-12-01
Thought: I now know the final answer.
Final Answer: No, there is no Prime member discount available for this specific smartwatch.

> Finished chain.
No, there is no Prime member discount available for this specific smartwatch.


In [50]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader

product_loader = CSVLoader('./data/realistic_digital_products.csv')
product_documents = product_loader.load()
product_text_splitter = CharacterTextSplitter(chunk_size=1024, separator="\n")
product_texts = product_text_splitter.split_documents(product_documents)
product_search = faiss.FAISS.from_documents(product_texts, OpenAIEmbeddings())
product_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=product_search, verbose=True)

@tool("FAQ")
def faq(intput: str) -> str:
    """"useful for when you need to answer questions about shopping policies, like exclusive deals, return policy, shipping policy, etc."""
    return faq_chain.run(intput)

@tool("Recommend Product")
def recommend_product(input: str) -> str:
    """"useful for when you need to search and recommend products and recommend it to the user"""
    return product_chain.run(input)

tools = [
    Tool(
        name = "Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    recommend_product, faq]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


/home/fw/miniconda3/envs/ai_aesthetics/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:251: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [51]:
question = "I want to buy two security cameras that need to be installed at my front door and living room. They should be controllable and viewable remotely via a smartphone. Can you help me with some recommendations?"
answer = agent.run(question)
print(answer)




> Entering new AgentExecutor chain...
 I need to find a product that meets the customer's needs.
Action: Recommend Product
Action Input: Security cameras that need to be installed at the front door and living room, controllable and viewable remotely via a smartphone

> Entering new VectorDBQA chain...

> Finished chain.

Observation:  You would need the Guardian Wi-Fi Security Camera, the CyberKey Smart Door Lock, and the Smart Home Hub Controller.
Thought: I now know the final answer.
Final Answer: We recommend the Guardian Wi-Fi Security Camera, the CyberKey Smart Door Lock, and the Smart Home Hub Controller for your security camera needs. They are controllable and viewable remotely via a smartphone.

> Finished chain.
We recommend the Guardian Wi-Fi Security Camera, the CyberKey Smart Door Lock, and the Smart Home Hub Controller for your security camera needs. They are controllable and viewable remotely via a smartphone.


In [64]:
import json

ORDER_1 = "20231127E45678"
ORDER_2 = "20231125C12345"

ORDER_1_DETAIL = {
    "order_number": ORDER_1,
    "status": "shipped",
    "shipping_date" : "2023-11-27",
    "estimated_delivered_date": "2023-12-01",
}

ORDER_2_DETAIL = {
    "order_number": ORDER_2,
    "status": "not Shipped",
    "shipping_date" : None,
    "estimated_delivered_date": None,
}

import re

@tool("Search Order")
def search_order(input:str)->str:
    """useful for when you need to answer questions about customers orders"""
    if input.strip() == ORDER_1:
        return json.dumps(ORDER_1_DETAIL)
    elif input.strip() == ORDER_2:
        return json.dumps(ORDER_2_DETAIL)
    else:
        return f"Sorry, we could not find your order based on {input}"

tools = [search_order,recommend_product, faq]
agent = initialize_agent(tools, llm=openai.OpenAI(temperature=0), agent="zero-shot-react-description", verbose=True)


In [65]:
question = "I have an order with the order number 20231127D3478E. The order status shows as not delivered. Could you please help me check it?"
answer = agent.run(question)
print(answer)




> Entering new AgentExecutor chain...
 I need to find out the status of the order
Action: Search Order
Action Input: 20231127D3478E
Observation: Sorry, we could not find your order based on 20231127D3478E
Thought: I need to find out more information about the order
Action: Search Order
Action Input: 20231127D3478E
Observation: Sorry, we could not find your order based on 20231127D3478E
Thought: I need to contact customer service
Action: FAQ
Action Input: order status

> Entering new VectorDBQA chain...

> Finished chain.

Observation:  FBA stands for Fulfillment by Amazon.
Thought: I now know the final answer
Final Answer: Please contact customer service for more information about the status of your order 20231127D3478E. FBA stands for Fulfillment by Amazon.

> Finished chain.
Please contact customer service for more information about the status of your order 20231127D3478E. FBA stands for Fulfillment by Amazon.


In [66]:
import re

@tool("Search Order")
def search_order(input:str)->str:
    """A tool that helps users check the latest order status and can handle the following situations:
1. When the user does not enter an order number, it will ask the user for the order number.
2. When the order number entered by the user cannot be found, it will ask the user to confirm whether the order number is correct for a second time.
"""
    pattern = r"^\d{8}[A-Z]\d{5}$"
    match = re.search(pattern, input)

    order_number = input
    if match:
        order_number = match.group(0)
    else:
        return "What is your order number, please?"
    if order_number == ORDER_1:
        return json.dumps(ORDER_1_DETAIL)
    elif order_number == ORDER_2:
        return json.dumps(ORDER_2_DETAIL)
    else:
        return f"Sorry, we could not find your order based on {input}."

tools = [search_order,recommend_product, faq]
agent = initialize_agent(tools, llm=openai.OpenAI(temperature=0), agent="zero-shot-react-description", verbose=True)

question = "I have an order with the order number 20231127D3478E. The order status shows as not delivered. Could you please help me check it?"
answer = agent.run(question)
print(answer)




> Entering new AgentExecutor chain...
 I should check the order status first.
Action: Search Order
Action Input: 20231127D3478E
Observation: Sorry, we could not find your order based on 20231127D3478E.
Thought: I should ask the user to confirm the order number.
Action: Search Order
Action Input: Confirm order number
Observation: What is your order number, please?
Thought: I should ask the user to enter the order number again.
Action: Search Order
Action Input: 20231127D3478E
Observation: Sorry, we could not find your order based on 20231127D3478E.
Thought: I should ask the user to confirm the order number again.
Action: Search Order
Action Input: Confirm order number
Observation: What is your order number, please?
Thought: I should ask the user to enter the order number one more time.
Action: Search Order
Action Input: 20231127D3478E
Observation: Sorry, we could not find your order based on 20231127D3478E.
Thought: I should ask the user to contact customer service.
Final Answer: We a

In [77]:
import re

answer_order_info = PromptTemplate(
    template="Please reply to the user with the following order information: \n\n {order}?", input_variables=["order"]
)
answer_order_llm = LLMChain(llm=ChatOpenAI(temperature=0),  prompt=answer_order_info)

@tool("Search Order", return_direct=True)
def search_order(input:str)->str:
    """useful for when you need to answer questions about customers orders"""
    pattern = r"^\d{8}[A-Z]\d{5}$"
    match = re.search(pattern, input)

    order_number = input
    if match:
        order_number = match.group(0)
    else:
        return "What is your order number, please?"
    if order_number == ORDER_1:
        return answer_order_llm.run(json.dumps(ORDER_1_DETAIL))
    elif order_number == ORDER_2:
        return answer_order_llm.run(json.dumps(ORDER_2_DETAIL))
    else:
        return f"Sorry, we could not find your order based on {input}."

from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

tools = [search_order,recommend_product, faq]
chatllm=ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation_agent = initialize_agent(tools, chatllm,
                                      agent="conversational-react-description",
                                      memory=memory, verbose=True)


In [68]:
question1 = "I bought a drone on November 20th, but I haven't received it yet. Can you help me check on it?"
answer1 = conversation_agent.run(question1)
print(answer1)




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search Order
Action Input: I bought a drone on November 20th
Observation: What is your order number, please?


> Finished chain.
What is your order number, please?


In [78]:
question2 = "My order number is 20231127E45678."
answer2 = conversation_agent.run(question2)
print(answer2)




> Entering new AgentExecutor chain...


Thought: Do I need to use a tool? Yes
Action: Search Order
Action Input: 20231127E45678
Observation: Dear user,

Thank you for your order. Here are the details:

Order Number: 20231127E45678
Status: Shipped
Shipping Date: 2023-11-27
Estimated Delivered Date: 2023-12-01

If you have any further questions or concerns, please feel free to ask.

Best regards,
[Your Name]


> Finished chain.
Dear user,

Thank you for your order. Here are the details:

Order Number: 20231127E45678
Status: Shipped
Shipping Date: 2023-11-27
Estimated Delivered Date: 2023-12-01

If you have any further questions or concerns, please feel free to ask.

Best regards,
[Your Name]


In [79]:
question3 = "What is Amazon's return policy?"
answer3 = conversation_agent.run(question3)
print(answer3)




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: FAQ
Action Input: What is Amazon's return policy?

> Entering new VectorDBQA chain...

> Finished chain.

Observation:  Amazon's return policy varies by item and seller, but in general, most items sold and fulfilled by Amazon can be returned within 30 days of receipt of shipment. For more information, please visit Amazon's Returns Support Center.
Thought:

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Do I need to use a tool? No`